In [7]:
# Print out GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

# Virtual Memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Fri Jun 10 15:39:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:18:00.0 Off |                  N/A |
|123%   83C    P2   189W / 250W |  11001MiB / 11264MiB |     89%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 27%   

In [4]:

# for data load
import datetime, os

# for reading and processing images
import imageio
from PIL import Image, ImageOps


from IPython.display import clear_output
# for visualizations
import matplotlib.pyplot as plt

import numpy as np # for using np arrays

# for bulding and running deep learning model
import tensorflow as tf
from tensorflow import keras as K


# print(f"Tensorflow ver. {tf.__version__}")


from tensorflow.keras.utils import get_source_inputs
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt
import pandas as pd
import time

# SM_FRAMEWORK=K
SM_FRAMEWORK=tf.keras
import segmentation_models as sm

sm.set_framework('tf.keras')
K.backend.set_image_data_format('channels_last')
from segmentation_models import get_preprocessing
seed=25
np.random.seed(25)
AUTOTUNE = tf.data.AUTOTUNE

In [5]:
BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)

In [8]:
model = sm.Unet(BACKBONE ,input_shape=(128, 128, 3),classes=10,activation='softmax', encoder_weights='imagenet')

2022-06-10 15:39:52.341178: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 19.66M (20611072 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-06-10 15:39:52.341808: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 19.66M (20611072 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-06-10 15:40:02.344093: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 19.66M (20611072 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-06-10 15:40:02.346105: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 19.66M (20611072 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-06-10 15:40:02.346170: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 576.0KiB (rounded to 589824)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_mallo

ResourceExhaustedError: failed to allocate memory [Op:AddV2]